In [92]:
# Ministry of Mines Auction Database URL
# url = 'https://mines.gov.in/webportal/content/archived/successful-auction-since-2015-6194' -- Depreciated
url = 'https://mines.gov.in/webportal/content/archived/sucessfully-auction-since-2015-6715'

In [93]:
# importing Library 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
from datetime import datetime
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
from dotenv import load_dotenv
import math

In [94]:
# triggering the chrome testing browser
chrome_options = Options()
chrome_options.use_chromium = True

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)

In [95]:
headers = ['state','name_of_the_block','mineral','auction_date', 'ml_or_cl', 'area_in_hectare',
          'final_bid_in_per_cent','preferred_bidder']

In [96]:
# collect all the block names from PostgreSQL
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2 import sql

load_dotenv(override=True)

database_url = os.getenv('DATABASE_URL')

try:
    connection = psycopg2.connect(database_url)
    cursor = connection.cursor()

    table_name = 'mom_auction'
    column_name = 'auction_date'
    query = sql.SQL("SELECT {column} from {table} order by {column} desc").format(
        column = sql.Identifier(column_name),
        table = sql.Identifier(table_name)
    )

    cursor.execute(query)

    rows = cursor.fetchall()
    # column_values = {row[0] for row in rows}
    # print(column_values)
    print(rows[0])

except Exception as e:
    print(f"Error: {e}")

finally:

    if cursor:
        cursor.close()
    if connection:
        connection.close()


(datetime.date(2024, 11, 20),)


In [97]:
# last date till scrapping has been done include
last_date = rows[0][0]
last_date

datetime.date(2024, 11, 20)

In [98]:
def convert_date(date_str):
    try:
        date_obj = datetime.strptime(date_str, '%d.%m.%Y')
        return date_obj.strftime('%Y-%m-%d')
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, '%d-%m-%Y')
            return date_obj.strftime('%Y-%m-%d')
        except ValueError:
            date_obj = datetime.strptime(date_str, '%d%m.%Y.')
            return date_obj.strftime('%Y-%m-%d')

In [107]:
last_date = datetime.strptime('2024-10-23', '%Y-%m-%d').date()
last_date

datetime.date(2024, 10, 23)

In [108]:
table_rows = driver.find_element(By.TAG_NAME,'tbody').find_elements(By.TAG_NAME, 'tr')
auction_data = []
cnt = 0
for row in table_rows[3:]:
    cells = []
    data = row.find_elements(By.TAG_NAME,'td')[1:]
    
    if len(data) == 9:        

        # converting auction date from DD.MM.YYYY format to YYYY-MM-DD format
        x = convert_date(data[3].text)

        
        # # if the date of the block is less than the last date till scrapping is done 
        # # we can avoid adding the data

        cnt+=1
        
        # if datetime.strptime(x,'%Y-%m-%d').date() < last_date:
        #     continue

        if datetime.strptime(x,'%Y-%m-%d').date() != last_date:
            continue

        print(x, last_date)
        cells.append(data[0].text)
        cells.append(data[1].text)
        cells.append(data[2].text)
        cells.append(x)
        cells.append(data[4].text)
        try:
            # Try to convert the text to a float
            value = float(data[5].text)
            cells.append(value)
        except ValueError:
            # If conversion fails, append NaN
            cells.append(np.nan)
        
        
        cells.append(float(data[7].text))
        cells.append(data[8].text)
        
        # cells = [data.text for data in row.find_elements(By.TAG_NAME,'td')[1:]]
        auction_data.append(cells)


2024-10-23 2024-10-23
2024-10-23 2024-10-23


In [109]:
len(auction_data)

2

In [110]:
df = pd.DataFrame(auction_data, columns = headers)

In [112]:
df = df.tail(1)

In [113]:
df

,state,name_of_the_block,mineral,auction_date,ml_or_cl,area_in_hectare,final_bid_in_per_cent,preferred_bidder
1,Central Government/Maharashtra,Vadakhol-Asoli Block,"Ni, Cr, Co and Associated Mineral\nMineral",2024-10-23,CL,1475.972,7.05,Assam Mineral Development Corporation Limited


,state,name_of_the_block,mineral,auction_date,ml_or_cl,area_in_hectare,final_bid_in_per_cent,preferred_bidder
2,Central Government/Jharkhand,Ponchi Block,Graphite,2024-10-22,ML,10.0,752.1,Satguru Mining Private Limited


In [114]:
database_url = os.getenv('DATABASE_URL')
try:
    connection = psycopg2.connect(database_url)
    cursor = connection.cursor()

    query = """
    INSERT INTO mom_auction(
        state,
        name_of_the_block,
        mineral,
        auction_date,
        ml_or_cl,
        area_in_hectare,
        final_bid_in_per_cent,
        preferred_bidder
    
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """

    # Iterate over DataFrame rows and insert data
    for index, row in df.iterrows():
        cursor.execute(query, (
            row['state'],
            row['name_of_the_block'],
            row['mineral'],
            row['auction_date'],  # Ensure auction_date is in the correct format
            row['ml_or_cl'],
            row['area_in_hectare'],
            row['final_bid_in_per_cent'],
            row['preferred_bidder']
        ))
    
    # Commit the transaction
    connection.commit()
    print("Data inserted successfully!")
except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


Data inserted successfully!
